In [1]:
import pandas as pd

## Model 1 - flat rate

In [22]:
total = pd.read_csv('./data/counts.csv')
print('total trips of everything: ', total['count'].sum())

total trips of everything:  113496874


In [56]:
#filter only pickup or dropoff in congestion pricing
zones = [4,12,13,43,45,48,50,68,79,87,88,90,100,107,113,114,125,137,140,141,142,143,144,148,158,161,162,163,164,170,186,209,211,224,229,230,231,232,233,234,236,237,238,239,246,249,261,262,263]

filtered_total = total[total['pickup_location_id'].isin(zones) | total['dropoff_location_id'].isin(zones)].copy()

#convert time time buckets 
# 8am Weekdays (6/6) - (morning rush-hours 6 - 10am) - 1
# 2pm Weekdays (6/6) - (non-rush hours) - 2
# 6pm Weekdays (6/6) - (night rush-hours 4 - 8pm) - 3
# 2pm Weekends (6/9) - (non-rush hours) - 0 

def getBucketId(row):
    if row['dow'] in (0, 6):
        return 0
    elif row['hour'] in (6,7,8,9,10):
        return 1
    elif row['hour'] in (4,5,6,7,8):
        return 3
    else: 
        return 2
    
filtered_total['bucketId'] = filtered_total.apply(getBucketId, axis = 1)

filtered_total.head()

,hour,dow,pickup_location_id,dropoff_location_id,count,bucketId
3,0,0,10,114,1,0
4,0,0,10,141,2,0
5,0,0,10,142,2,0
6,0,0,10,144,1,0
8,0,0,10,161,1,0


In [57]:
print('total trips of filtered congrestion zones', filtered_total['count'].sum())

total trips of filtered congrestion zones 105334092


In [58]:
105334092 * 2.5

263335230.0

## Setup Model 2 and 3

In [59]:
legs = pd.read_csv('./data/legs.csv')
legs.head()

,_uid_,fromid,toid,timebucket,distance_i,duration_i
0,1,1,4,0,4258.536898,1113.309588
1,2,1,4,1,4258.536898,1113.309588
2,3,1,4,2,4258.536898,1113.309588
3,4,1,4,3,4258.536898,1113.309588
4,5,1,12,0,92.058823,5.942862


In [60]:
#join filtered_total by time with legs
join = filtered_total.merge(
                    legs, 
                    left_on=['bucketId','pickup_location_id','dropoff_location_id'],
                    right_on=['timebucket','fromid','toid']
                    )
join.head()

,hour,dow,pickup_location_id,dropoff_location_id,count,bucketId,_uid_,fromid,toid,timebucket,distance_i,duration_i
0,0,0,10,114,1,0,2628,10,114,0,4148.585834,953.916215
1,0,6,10,114,1,0,2628,10,114,0,4148.585834,953.916215
2,1,6,10,114,1,0,2628,10,114,0,4148.585834,953.916215
3,6,0,10,114,1,0,2628,10,114,0,4148.585834,953.916215
4,8,0,10,114,1,0,2628,10,114,0,4148.585834,953.916215


## Model 2 - Dwell Time in Zone

In [61]:
#get the total dwell time in zone by mins
total_mins = sum(join['count'] * (join['duration_i'] / 60))

In [62]:
total_mins  * 0.30 

172918569.36927497

## Model 3 - Distance in Zone

In [63]:
#get the total distance in zone by miles
total_miles = sum(join['count'] * (join['distance_i']/ 1609.344))

In [64]:
total_miles

96325055.85633157

# Time Bucket based modified models

# Model 1a - time based rates, rushhour, weekday and weekends

In [73]:
join.groupby('bucketId').count()

,hour,dow,pickup_location_id,dropoff_location_id,count,_uid_,fromid,toid,timebucket,distance_i,duration_i
bucketId,,,,,,,,,,,
0,201578,201578,201578,201578,201578,201578,201578,201578,201578,201578,201578
1,87476,87476,87476,87476,87476,87476,87476,87476,87476,87476,87476
2,339175,339175,339175,339175,339175,339175,339175,339175,339175,339175,339175
3,31586,31586,31586,31586,31586,31586,31586,31586,31586,31586,31586


In [68]:
#sum in counts by buckets

# 8am Weekdays (6/6) - (morning rush-hours 6 - 10am) - 1 - $5
# 2pm Weekdays (6/6) - (non-rush hours) - 2 -  $2.5
# 6pm Weekdays (6/6) - (night rush-hours 4 - 8pm) - 3 $5
# 2pm Weekends (6/9) - (non-rush hours) - 0 - $2

rate = {
    1: 5.0,
    2: 2.5,
    3: 5.0,
    0: 2.0
}

def getTimeBasedRateRev(row):
    return rate[row['bucketId']] * row['count']


sum(join.apply(getTimeBasedRateRev, axis = 1))

138100113.0

## Model 2a - Dwell Time based on time

In [78]:
rate = {
    1: 0.50,
    2: 0.25,
    3: 0.50,
    0: 0.20
}

def getTimeBasedDwellRev(row):
    return rate[row['bucketId']] * row['count'] * (row['duration_i'] / 60)

sum(join.apply(getTimeBasedDwellRev, axis = 1))

160287460.7156365

## Model 3a - Distance in Zone based on time

In [79]:
rate = {
    1: 1.50,
    2: 1.00,
    3: 1.50,
    0: 0.75
}

def getTimeBasedDistanceRev(row):
    return rate[row['bucketId']] * row['count'] * (row['distance_i'] / 1609.344)

sum(join.apply(getTimeBasedDistanceRev, axis = 1))

97118741.39256494

time bucket based |rate   | duration in zone  | distance in zone |
--- | --- | --- | --- |
no | `$263,335,230.00` |  `$172,918,569.37` | `$96,325,055.86` |
yes | `$138,100,113.00` | `$160,287,460.72` | `$97,118,741.39`